In [1]:
import os
# os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
import pandas as pd

In [2]:
import tensorflow as tf

In [3]:
gpus=tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [4]:
directory="D:/Course/DrivenData Project/Dataset/"
df=pd.read_csv(directory+'train_features.csv')
df

,id,filepath,site
0,ZJ000000,train_features/ZJ000000.jpg,S0120
1,ZJ000001,train_features/ZJ000001.jpg,S0069
2,ZJ000002,train_features/ZJ000002.jpg,S0009
3,ZJ000003,train_features/ZJ000003.jpg,S0008
4,ZJ000004,train_features/ZJ000004.jpg,S0036
...,...,...,...
16483,ZJ016483,train_features/ZJ016483.jpg,S0093
16484,ZJ016484,train_features/ZJ016484.jpg,S0043
16485,ZJ016485,train_features/ZJ016485.jpg,S0089
16486,ZJ016486,train_features/ZJ016486.jpg,S0095


In [5]:
import numpy as np
train_labels = pd.read_csv(directory+'train_labels.csv')
classes = train_labels.columns[1:]
train_labels = train_labels.iloc[:, 1:]
train_y = np.array([np.argmax(train_labels.iloc[i, :]) for i in range(train_labels.shape[0])])
l=list(train_y)
l

[1,
 6,
 1,
 6,
 5,
 4,
 3,
 0,
 5,
 1,
 5,
 5,
 6,
 2,
 6,
 2,
 5,
 1,
 2,
 6,
 5,
 6,
 0,
 5,
 6,
 7,
 6,
 6,
 6,
 2,
 0,
 6,
 4,
 3,
 6,
 7,
 3,
 0,
 6,
 0,
 3,
 2,
 5,
 0,
 3,
 2,
 5,
 6,
 6,
 0,
 7,
 2,
 4,
 5,
 7,
 6,
 3,
 0,
 6,
 6,
 1,
 2,
 2,
 5,
 2,
 0,
 3,
 0,
 7,
 3,
 2,
 1,
 4,
 3,
 1,
 3,
 7,
 3,
 0,
 1,
 6,
 1,
 2,
 3,
 2,
 3,
 5,
 1,
 0,
 6,
 5,
 2,
 1,
 3,
 0,
 7,
 5,
 5,
 0,
 5,
 2,
 0,
 5,
 6,
 2,
 6,
 5,
 2,
 5,
 4,
 2,
 0,
 7,
 3,
 1,
 3,
 3,
 6,
 7,
 5,
 6,
 6,
 7,
 0,
 0,
 1,
 7,
 0,
 5,
 1,
 7,
 0,
 3,
 3,
 4,
 3,
 3,
 0,
 5,
 3,
 0,
 2,
 2,
 7,
 2,
 0,
 7,
 7,
 7,
 0,
 0,
 3,
 0,
 7,
 1,
 2,
 5,
 0,
 3,
 5,
 7,
 5,
 1,
 1,
 6,
 2,
 5,
 1,
 2,
 7,
 0,
 6,
 2,
 6,
 1,
 5,
 3,
 6,
 3,
 5,
 2,
 2,
 6,
 6,
 7,
 5,
 3,
 0,
 0,
 5,
 1,
 0,
 1,
 2,
 6,
 2,
 2,
 0,
 7,
 6,
 2,
 6,
 3,
 7,
 2,
 7,
 7,
 0,
 1,
 7,
 0,
 0,
 7,
 5,
 6,
 6,
 0,
 7,
 3,
 7,
 4,
 2,
 0,
 7,
 3,
 7,
 7,
 6,
 0,
 5,
 6,
 1,
 2,
 6,
 3,
 5,
 5,
 5,
 0,
 0,
 3,
 2,
 3,
 7,
 5,
 1,
 6,
 0,
 6,
 6,


In [6]:
df['label']=train_y
df

,id,filepath,site,label
0,ZJ000000,train_features/ZJ000000.jpg,S0120,1
1,ZJ000001,train_features/ZJ000001.jpg,S0069,6
2,ZJ000002,train_features/ZJ000002.jpg,S0009,1
3,ZJ000003,train_features/ZJ000003.jpg,S0008,6
4,ZJ000004,train_features/ZJ000004.jpg,S0036,5
...,...,...,...,...
16483,ZJ016483,train_features/ZJ016483.jpg,S0093,2
16484,ZJ016484,train_features/ZJ016484.jpg,S0043,5
16485,ZJ016485,train_features/ZJ016485.jpg,S0089,3
16486,ZJ016486,train_features/ZJ016486.jpg,S0095,1


In [7]:
filepaths=df['filepath'].values
labels=df['label'].values
ds_train=tf.data.Dataset.from_tensor_slices((filepaths,labels))
len(ds_train)

16488

In [10]:
type(ds_train)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [9]:
labels

array([1, 6, 1, ..., 3, 1, 3], dtype=int64)

In [11]:
def read_image(image_file,label):
    image=tf.io.read_file(directory+image_file)
    image=tf.image.decode_image(image,channels=1,dtype=tf.float32,expand_animations = False)
    return image,label

In [12]:
def resize_and_rescale(image,label):
    image=tf.cast(image,tf.float32)
    image=tf.image.resize(image,[224,224])
    return image,label

In [13]:
import tensorflow_io as tfio
def grey(image,label):
#     image=tfio.experimental.color.bgr_to_rgb(image)
    image=(image/255.0)
    return image,label

ModuleNotFoundError: No module named 'tensorflow_io'

In [ ]:
ds_train=ds_train.map(read_image).map(resize_and_rescale).map(grey).batch(32)
len(ds_train)

In [ ]:
for i in ds_train:
    print (i)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, MaxPool2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.regularizers import l1, l2, l1_l2

In [ ]:
model=Sequential()
model.add(Conv2D(60,(3,3),input_shape=(224,224,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(60,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(100,activation='relu',activity_regularizer=l1(0.001)))
model.add(Dense(1))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='Adam' ,metrics=['accuracy'])

In [ ]:
model.fit(ds_train,epochs=5,verbose=1)